#### PyTorch has emerged as a major contender in the race to be the king of deep learning frameworks.

In this notebook I will go over some regular snippets and techniques of it.

# Compute basic gradients from the sample tensors using PyTorch

#### First some basics of Pytorch here

**Autograd**: This class is an engine to calculate derivatives (Jacobian-vector product to be more precise). It records a graph of all the operations performed on a gradient enabled tensor and creates an acyclic graph called the dynamic computational graph. The leaves of this graph are input tensors and the roots are output tensors. Gradients are calculated by tracing the graph from the root to the leaf and multiplying every gradient in the way using the chain rule.

A Variable class wraps a tensor. You can access this tensor by calling `.data` attribute of a Variable.

The Variable also stores the gradient of a scalar quantity (say, loss) with respect to the parameter it holds. This gradient can be accessed by calling the `.grad` attribute. This is basically the gradient computed up to this particular node, and the gradient of the every subsequent node, can be computed by multiplying the edge weight with the gradient computed at the node just before it.

The third attribute a Variable holds is a grad_fn, a Function object which created the variable.

**Variable**: The Variable, just like a Tensor is a class that is used to hold data. It differs, however, in the way it’s meant to be used. Variables are specifically tailored to hold values which change during training of a neural network, i.e. the learnable paramaters of our network. Tensors on the other hand are used to store values that are not to be learned. For example, a Tensor maybe used to store the values of the loss generated by each example.

Every **variable** object has several members one of them is **grad**:

**grad**: grad holds the value of gradient. If requires_grad is False it will hold a None value. Even if requires_grad is True, it will hold a None value unless .backward() function is called from some other node. For example, if you call out.backward() for some variable out that involved x in its calculations then x.grad will hold ∂out/∂x.

**Backward() function**
Backward is the function which actually calculates the gradient by passing it’s argument (1x1 unit tensor by default) through the backward graph all the way up to every leaf node traceable from the calling root tensor. The calculated gradients are then stored in .grad of every leaf node. Remember, the backward graph is already made dynamically during the forward pass. Backward function only calculates the gradient using the already made graph and stores them in leaf nodes.

In [1]:
import torch
from torch.autograd import Variable

def forward(x):
    return x * w

w = Variable(torch.Tensor([1.0]), requires_grad=True)
# . On setting .requires_grad = True they start forming a backward graph
# that tracks every operation applied on them to calculate the gradients
# using something called a dynamic computation graph (DCG)
# When you finish your computation you can call .backward() and have
# all the gradients computed automatically. The gradient for this tensor
# will be accumulated into .grad attribute.

# Now create an array of data.
# By PyTorch’s design, gradients can only be calculated
# for floating point tensors which is why I’ve created a float type
# array before making it a gradient enabled PyTorch tensor
x_data = [11.0, 22.0, 33.0]
y_data = [21.0, 14.0, 64.0]

def loss_function(x, y):
    y_pred = forward(x)
    return (y_pred - y) * (y_pred - y)


# Now running the training loop
for epoch in range(10):
    for x_val, y_val in zip(x_data, y_data):
        l = loss_function(x_val, y_val)
        l.backward()
        print("\tgrad: ", x_val, y_val, w.grad.data[0])
        w.data = w.data - 0.01 * w.grad

        # Manually set the gradient to zero after updating weights
        w.grad.data.zero_()

        print('progress: ', epoch, l.data[0])

	grad:  11.0 21.0 tensor(-220.)
progress:  0 tensor(100.)
	grad:  22.0 14.0 tensor(2481.6001)
progress:  0 tensor(3180.9602)
	grad:  33.0 64.0 tensor(-51303.6484)
progress:  0 tensor(604238.8125)
	grad:  11.0 21.0 tensor(118461.7578)
progress:  1 tensor(28994192.)
	grad:  22.0 14.0 tensor(-671630.6875)
progress:  1 tensor(2.3300e+08)
	grad:  33.0 64.0 tensor(13114108.)
progress:  1 tensor(3.9481e+10)
	grad:  11.0 21.0 tensor(-30279010.)
progress:  2 tensor(1.8943e+12)
	grad:  22.0 14.0 tensor(1.7199e+08)
progress:  2 tensor(1.5279e+13)
	grad:  33.0 64.0 tensor(-3.3589e+09)
progress:  2 tensor(2.5900e+15)
	grad:  11.0 21.0 tensor(7.7553e+09)
progress:  3 tensor(1.2427e+17)
	grad:  22.0 14.0 tensor(-4.4050e+10)
progress:  3 tensor(1.0023e+18)
	grad:  33.0 64.0 tensor(8.6030e+11)
progress:  3 tensor(1.6991e+20)
	grad:  11.0 21.0 tensor(-1.9863e+12)
progress:  4 tensor(8.1519e+21)
	grad:  22.0 14.0 tensor(1.1282e+13)
progress:  4 tensor(6.5750e+22)
	grad:  33.0 64.0 tensor(-2.2034e+14)
pro

Weight initialization is an important task in training a neural network,
whether its a convolutional neural network
(CNN), a deep neural network (DNN), and a recurrent neural network
(RNN). Lets some examples of initializing the weights.


Weight initialization can be done by using various methods, including
random weight initialization.
Weight initialization based on a distribution
is done using
- Uniform distribution,
- Bernoulli distribution,
- Multinomial distribution, and normal distribution.

To execute a neural network, a set of initial weights needs to be passed to
the backpropagation layer to compute the loss function (and hence, the
accuracy can be calculated). The selection of a method depends on the
data type, the task, and the optimization required for the model.

Bernoulli Distribution is a random experiment that has only two outcomes (usually called a “Success” or a “Failure”). It is best used when we have two outcomes of a given event. Its considered as the discrete
probability distribution, which has two possible outcomes. If the event happens, then the value is 1, and if the event does not happen, then the value is 0.

For discrete probability distribution, we calculate probability mass
function instead of probability density function. The probability mass
function looks like the following formula.

![](https://i.imgur.com/bz2dWtc.png)

From the Bernoulli distribution, we create sample tensors by considering the uniform distribution of size 4 and 4 in a matrix format, as follows.

 Specifically, `torch.bernoulli()` samples from the distribution and returns a binary value (i.e. either 0 or 1). Here, it returns 1 with probability p and return 0 with probability 1-p.

```python
torch.bernoulli(input, *, generator=None, out=None)
```
It draws binary random numbers (0 or 1) from a Bernoulli distribution.

Syntax

```python
torch.bernoulli(input, *, generator=None, out=None) → Tensor
```

Parameters :

input (Tensor) – the input tensor of probability values for the Bernoulli distribution

generator (torch.Generator, optional) – a pseudorandom number generator for sampling

out (Tensor, optional) – out tensor only has values 0 or 1 and is of the same shape as input.

The input tensor should be a tensor containing probabilities to be used for drawing the binary random number. Hence, all values in input have to be in the range:

0 <= input_i <=1


In [2]:
torch.bernoulli(torch.Tensor(4, 4).uniform_(0, 1))


tensor([[0., 0., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 0., 1.],
        [1., 1., 0., 1.]])

# Generation of sample random values from a multinomial distribution

Note the syntax of multinomial function from official doc

```python
torch.multinomial(input, num_samples, replacement=False, *, generator=None, out=None) → LongTensor
```
Returns a tensor where each row contains num_samples indices sampled from the multinomial probability distribution located in the corresponding row of tensor input.



In [3]:
sample_tensor = torch.Tensor([10, 10, 13, 10, 34,45,65,67,87,89,87,34])
torch.multinomial(torch.tensor([10., 10., 13., 10., 34., 45., 65., 67., 87., 89., 87., 34.]), 3)

tensor([ 8, 10, 11])

Sampling from multinomial distribution with a replacement returns the tensors’ index values.

In [4]:
torch.multinomial(torch.tensor([10., 10., 13., 10., 34., 45., 65., 67., 87., 89., 87., 34.]), 5, replacement=True)

tensor([10,  8,  8,  0,  8])

And now, the weight initialization from the normal distribution, which is also a method
that is used in fitting a neural network, fitting a deep neural network, and
CNN and RNN. Let’s have a look at the process of creating a set of random
weights generated from a normal distribution.

Syntax

```python
torch.normal(mean, std, *, generator=None, out=None) → Tensor
```
Returns a tensor of random numbers drawn from separate normal distributions whose mean and standard deviation are given.

The mean is a tensor with the mean of each output element’s normal distribution

The std is a tensor with the standard deviation of each output element’s normal distribution

The shapes of mean and std don’t need to match, but the total number of elements in each tensor need to be the same.

In [5]:
torch.normal(mean=torch.arange(1., 11), std=torch.arange(1, 0, -0.1))

tensor([ 0.4350,  1.3608,  2.3142,  4.1479,  4.6382,  5.9646,  6.9279,  7.9011,
         8.9942, 10.0591])

In [6]:
torch.normal(mean=0.5, std=torch.arange(1.,6.))

tensor([ 0.3024,  1.6342,  3.4563, -2.8438,  3.4015])

In [7]:
torch.normal(mean=0.5, std=torch.arange(0.2, 0.6))


tensor([0.5592])

# Variable in PyTorch and its defined? What is a random variable in PyTorch?

In PyTorch, the algorithms are represented as a computational graph.

A variable is considered as a representation around the tensor object,
corresponding gradients (slope of the function), and a reference to the function from where it was
created. 

The slope of the function can be computed by the derivative of the
function with respect to the parameters that are present in the function.

Basically, a PyTorch variable is a node in a computational graph, which
stores data and gradients. When training a neural network model, after
each iteration, we need to compute the gradient of the loss function with
respect to the parameters of the model, such as weights and biases. After
that, we usually update the weights using the gradient descent algorithm.

Below Figure explains how the linear regression equation is deployed under
the hood using a neural network model in the PyTorch framework.
In a computational graph structure, the sequencing and ordering
of tasks is very important. The one-dimensional tensors are X, Y, W,
and alpha. The direction of the arrows change when we
implement backpropagation to update the weights to match with Y, so that
the error or loss function between Y and predicted Y can be minimized.


![Imgur](https://imgur.com/6JOtOGb.png)

#### Lets see and example

An example of how a variable is used to create a computational graph is
displayed in the following script. There are three variable objects around
tensors— x1, x2, and x3—with random points generated from a = 12 and
b = 23. The graph computation involves only multiplication and addition,
and the final result with the gradient is shown.

The partial derivative of the loss function with respect to the weights
and biases in a neural network model is achieved in PyTorch using the
Autograd module. Variables are specifically designed to hold the changed
values while running a backpropagation in a neural network model when
the parameters of the model change. The variable type is just a wrapper
around the tensor. It has three properties: data, grad, and function.





In [8]:
from torch.autograd import Variable
Variable(torch.ones(2,2), requires_grad=True)


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [9]:
a, b = 12, 23
x1 = Variable(torch.randn(a, b), requires_grad=True )
x2 = Variable(torch.randn(a,b), requires_grad=True)
x3 = Variable(torch.randn(a,b), requires_grad=True)

In [10]:
c = x1 * x2
d = a + x3
e = torch.sum(d)

e.backward()

print(e)

tensor(3305.0916, grad_fn=<SumBackward0>)


In [11]:
x1.data

tensor([[-0.5805, -1.6211, -0.0220,  0.1063,  0.6003,  1.1850,  0.5650, -0.4607,
          1.0795, -0.2618, -0.5876,  0.9575, -0.0534,  1.2101,  1.3989, -0.0707,
         -0.6653, -0.6698, -0.5093, -0.1171, -0.0559, -0.6459,  0.7618],
        [-1.6172, -1.4776, -1.3159, -0.7741,  0.2111,  1.6640,  0.1075,  0.8687,
          0.0398,  0.4285,  0.1697,  0.4668, -0.3620,  0.7425,  0.4021,  0.4445,
          1.7736, -0.5256,  1.9984,  1.0971, -0.2418, -0.9940,  0.4842],
        [-1.2023,  0.3436,  0.0941, -1.2934, -1.0873,  0.4688,  1.3987, -1.3774,
          0.1682,  0.8509, -0.7933,  0.4016, -0.2627, -1.2704, -0.6581, -0.6152,
         -1.7383, -0.7637,  0.4793, -0.1167, -0.4131,  0.6578, -1.4892],
        [-0.5291,  0.2822, -0.2390, -0.5625, -1.0537, -0.3463, -1.3984,  0.7116,
          0.6512,  1.7050,  0.7495,  0.5477, -0.0778, -0.1877,  0.5774, -0.5838,
         -0.5228, -1.5167,  1.2894, -0.7242,  0.3746,  1.9546, -0.7291],
        [-1.2382, -1.7610,  0.3786, -0.7586, -1.4637, -0.382

In [12]:
x2.data

tensor([[ 1.5597, -0.1538,  0.1115,  1.0113, -0.2525, -2.3110,  2.0029, -0.2659,
         -0.7239, -0.5386,  1.1574,  0.7259,  1.2875, -0.1156,  0.4462, -0.0466,
         -0.2411, -0.3121,  0.3035, -0.0231,  0.1575,  0.0137,  1.4618],
        [-2.6362, -0.1665, -0.3811,  1.4514,  0.9064, -0.7623,  1.9534,  1.1944,
          0.6413, -0.7048, -0.4018, -0.5417, -0.9185,  2.3222, -0.8024,  0.3821,
          0.1069, -0.7797,  1.8992,  0.1724, -1.9079,  1.3650,  0.1518],
        [-1.8273,  0.4821, -0.0423,  1.3044,  0.7008,  0.9906, -1.9159, -0.0876,
          0.5237, -0.2249, -0.8327,  0.3556,  0.4029,  0.9445, -1.0837, -0.9799,
         -0.5934, -1.1064, -0.9814,  1.7679, -0.3688,  1.0304,  0.7488],
        [ 0.0643, -2.3823,  0.4109,  1.2474,  0.1031, -1.2063, -0.4925, -1.0155,
         -0.0631,  1.8255,  0.7474, -0.3815,  0.4656,  0.2265, -0.0691,  0.9685,
         -0.7942, -0.0685, -0.6331,  0.5485,  0.1820, -0.8391,  0.0730],
        [-1.3950,  0.6110,  1.1048,  0.0879,  0.3168, -0.454

In [13]:
x3.data

tensor([[ 1.5732e+00, -1.8919e+00, -9.5864e-01,  1.0764e+00, -1.1491e+00,
          1.2582e+00,  1.3774e+00, -7.9841e-01,  4.1802e-01,  1.3354e+00,
         -2.2857e+00,  1.2406e+00,  3.2930e-01,  2.7476e-01, -7.1076e-01,
         -7.8910e-01, -1.3080e-01,  4.5392e-01,  8.6285e-01,  8.0801e-01,
          5.4228e-01,  1.6918e-01, -3.9944e-01],
        [ 2.3779e-01, -1.1615e-01, -5.4451e-01,  1.4247e+00,  7.1588e-01,
          1.5356e+00,  4.4198e-03,  3.7880e-01, -7.9946e-01,  3.4540e-01,
          1.4109e+00,  1.4679e-02, -1.4088e+00,  6.4824e-01, -8.7981e-01,
         -1.8292e+00,  1.2675e+00,  4.0991e-01, -1.9237e+00,  1.0276e+00,
         -5.1640e-01, -3.6208e-01,  1.9239e+00],
        [-5.2775e-01,  1.9665e-02, -1.2310e+00, -2.2846e-01,  5.6001e-01,
          1.5126e-01,  3.8631e+00,  7.2139e-01, -2.4427e+00, -3.1431e-01,
          1.0223e+00,  1.8373e+00, -1.5903e+00,  8.1811e-01, -5.5065e-01,
          1.9191e-01,  4.3877e-01,  9.3067e-01,  7.9798e-02,  1.3302e+00,
         -4.70

# How do we set up a loss function and optimize it ? 

Choosing the right loss function increases the chances of model convergence. 

we use another tensor as the update variable, and introduce
the tensors to the sample model and compute the error or loss. Then we
compute the rate of change in the loss function to measure the choice of
loss function in model convergence.

In the following example, t_c and t_u are two tensors. This can be
constructed from any NumPy array.


In [14]:
torch.__version__

'1.7.1'

In [15]:
torch.tensor

<function _VariableFunctionsClass.tensor>

The sample model is just a linear equation to make the calculation
happen and the loss function defined if the mean square error (MSE)
shown next. For now, this is just a simple linear equation
computation.


In [16]:
#height of people
t_c = torch.tensor([58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0])

#weight of people
t_u = torch.tensor([115.0, 117.0, 120.0, 123.0, 126.0, 129.0, 132.0, 135.0, 139.0, 142.0, 146.0, 150.0, 154.0, 159.0,164.0])



Let’s now define the model. The w parameter is the weight tensor,
which is multiplied with the t_u tensor. The result is added with a constant
tensor, b, and the loss function chosen is a custom-built one; it is also available in PyTorch. 

In the following example, t_u is the tensor used, t_p
is the tensor predicted, and t_c is the precomputed tensor, with which the
predicted tensor needs to be compared to calculate the loss function.

The formula $$w * t_u + b$$ is the linear equation representation of a
tensor-based computation.



In [17]:
def model(t_u, w, b):
    return w * t_u + b
  
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()
  
w = torch.ones(1)
b = torch.zeros(1)

t_p = model(t_u, w, b)
t_p

tensor([115., 117., 120., 123., 126., 129., 132., 135., 139., 142., 146., 150.,
        154., 159., 164.])

In [18]:
loss = loss_fn(t_p, t_c)
loss

tensor(5259.7334)

The initial loss value is 5259.7334, which is too high because of the
initial round of weights chosen. The error in the first round of iteration
is backpropagated to reduce the errors in the second round, for which
the initial set of weights needs to be updated. Therefore, the rate of
change in the loss function is essential in updating the weights in the
estimation process.


In [19]:
delta = 0.1

loss_rate_of_change_w = (loss_fn(model(t_u, 
                                       w + delta, b), 
                                 t_c) - loss_fn(model(t_u, w - delta, b), 
                                                t_c)) / (2.0 * delta)

In [20]:
learning_rate = 1e-2

w = w - learning_rate * loss_rate_of_change_w

There are two parameters to update the rate of loss function: the
learning rate at the current iteration and the learning rate at the previous
iteration. If the delta between the two iterations exceeds a certain
threshold, then the weight tensor needs to be updated, else model
convergence could happen. The preceding script shows the delta and
learning rate values. Currently, these are static values that the user has the
option to change.


In [21]:
loss_rate_of_change_b = (loss_fn(model(t_u, w, b + delta), t_c) - 
                         loss_fn(model(t_u, w, b - delta), t_c)) / (2.0 * delta)

b = b - learning_rate * loss_rate_of_change_b

b

tensor([544.])

This is how a simple mean square loss function works in a two-­
dimensional tensor example, with a tensor size of 10,5.
Let’s look at the following example. The MSELoss function is within the
neural network module of PyTorch.


In [22]:
from torch import nn
loss = nn.MSELoss()
input = torch.randn(10, 5, requires_grad=True)
target = torch.randn(10, 5)
output = loss(input, target)
output.backward()

When we look at the gradient calculation that is used for
backpropagation, it is shown as MSELoss.


In [23]:
output.grad_fn

# Tensor differentiation and its relevance in computational graph execution using the PyTorch framework

The computational graph network is represented by nodes and connected
through functions. There are two different kinds of nodes: dependent and
independent. Dependent nodes are waiting for results from other nodes
to process the input. Independent nodes are connected and are either
constants or the results. Tensor differentiation is an efficient method to
perform computation in a computational graph environment.

In a computational graph, tensor differentiation is very effective because
the tensors can be computed as parallel nodes, multiprocess nodes, or
multithreading nodes. The major deep learning and neural computation
frameworks include this tensor differentiation.
Autograd is the function that helps perform tensor differentiation,
which means calculating the gradients or slope of the error function,
and backpropagating errors through the neural network to fine-tune the
weights and biases. Through the learning rate and iteration, it tries to
reduce the error value or loss function.
To apply tensor differentiation, the nn.backward() method needs to
be applied. Let’s take an example and see how the error gradients are
backpropagated. To update the curve of the loss function, or to find where
the shape of the loss function is minimum and in which direction it is
moving, a derivative calculation is required. Tensor differentiation is a way
to compute the slope of the function in a computational graph.


In [24]:
# Make a sample tensor x, for which automatic gradient calculation needs to happen.
x = Variable(torch.ones(4, 4) * 12.5, requires_grad=True)
x

tensor([[12.5000, 12.5000, 12.5000, 12.5000],
        [12.5000, 12.5000, 12.5000, 12.5000],
        [12.5000, 12.5000, 12.5000, 12.5000],
        [12.5000, 12.5000, 12.5000, 12.5000]], requires_grad=True)

In [25]:
#  Create a linear function fn that is created using the x variable.
fn = 2 * (x * x) + 5 * x + 6


#  Using the backward function, we can perform a backpropagation calculation. 
fn.backward(torch.ones(4,4))

# The .grad() function holds the final output from the tensor differentiation.
x.grad

tensor([[55., 55., 55., 55.],
        [55., 55., 55., 55.],
        [55., 55., 55., 55.],
        [55., 55., 55., 55.]])